# Converting tiff to geojson

In [10]:
import geopandas as gpd
import rasterio

In [5]:
tiff_name = 'DATA/esp_ppp_2020_constrained.tif'

In [6]:
data = rasterio.open(tiff_name).meta
data

{'driver': 'GTiff',
 'dtype': 'float32',
 'nodata': -99999.0,
 'width': 26989,
 'height': 19386,
 'count': 1,
 'crs': CRS.from_epsg(4326),
 'transform': Affine(0.0008333333299862907, 0.0, -18.162083246,
        0.0, -0.0008333333299803984, 43.792083359)}

In [7]:
c = str(data['crs'])
c_s = c.split(':')
c_s[1]

'4326'

In [8]:
from rasterio.features import shapes
mask = None
with rasterio.open(tiff_name) as src:
    image = src.read(1)
    results = (
    {'properties': {'NDVI': v}, 'geometry': s}
    for i, (s, v) 
    in enumerate(
        shapes(image, mask=mask, transform=data['transform'])))

In [9]:
geoms = list(results)

In [11]:
gpd_polygonized_raster = gpd.GeoDataFrame.from_features(geoms, crs=c)

In [13]:
gpd_polygonized_raster = gpd_polygonized_raster[gpd_polygonized_raster['NDVI']>0]

In [14]:
gpd_polygonized_raster

,geometry,NDVI
0,"POLYGON ((-7.68958 43.79042, -7.68958 43.78958...",1.469323
1,"POLYGON ((-7.68875 43.79042, -7.68875 43.78958...",1.455587
2,"POLYGON ((-7.68792 43.79042, -7.68792 43.78958...",1.459332
3,"POLYGON ((-7.68875 43.78958, -7.68875 43.78875...",1.472362
4,"POLYGON ((-7.68792 43.78958, -7.68792 43.78875...",1.660093
...,...,...
3464583,"POLYGON ((-17.98292 27.63958, -17.98292 27.638...",3.980716
3464584,"POLYGON ((-17.98125 27.63958, -17.98125 27.638...",4.056968
3464585,"POLYGON ((-17.98042 27.63958, -17.98042 27.638...",4.142812
3464586,"POLYGON ((-17.97958 27.63958, -17.97958 27.638...",4.632590


In [15]:
crs_sys = 'epsg:'+c_s[1]

In [16]:
gpd_polygonized_raster['geometry'] = gpd_polygonized_raster['geometry'].to_crs({'init': crs_sys})

/home/a_romegar/anaconda3/envs/ironhack/lib/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/a_romegar/anaconda3/envs/ironhack/lib/python3.8/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [17]:
gpd_polygonized_raster.to_file('DATA/esptotal.geojson', driver='GeoJSON') 